In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
pickles_path = "../../../pickles/"

In [3]:
# unpickle
with open(pickles_path + "en_1218_traffic_summary_by_type.pkl", 'rb') as picklefile: 
    traffic_summary_by_type = pickle.load(picklefile)

traffic_summary_by_type

,edges_count,traffic_volume,edges_count_pct,traffic_volume_pct,traffic_avg
type,,,,,
external,10990982,5100299091,0.368235,0.741460,464.043985
link,17851574,1718399222,0.598088,0.249814,96.260376
other,1005180,60022438,0.033677,0.008726,59.713124


In [4]:
# unpickle
with open(pickles_path + "en_1218_external_traffic_summary_by_cat.pkl", 'rb') as picklefile: 
    external_traffic_summary_by_cat = pickle.load(picklefile)

external_traffic_summary_by_cat

,edges_count,traffic_volume,edges_count_pct,traffic_volume_pct,traffic_avg
prev,,,,,
other-empty,5093433,1779965188,0.463419,0.348992,349.462767
other-external,787311,116732857,0.071632,0.022887,148.267784
other-internal,1348892,132349844,0.122727,0.025949,98.117450
other-other,374921,36972380,0.034112,0.007249,98.613788
other-search,3386425,3034278822,0.308109,0.594922,896.012409


From the [EDA notebook](English_Wikipedia_EDA.ipynb#Distribution-of-traffic-volume-and-edge-counts-across-the-3-reference-types):  

number of terminal nodes in link edges = 2598302  
number of terminal nodes in internal search edges = 414847

### Data for the pie/donut chart segments  

In [5]:
traffic_by_type = external_traffic_summary_by_cat[["traffic_volume", "edges_count"]].copy()
traffic_by_type.sort_values("traffic_volume", ascending=False, inplace=True)

temp = traffic_summary_by_type[["traffic_volume", "edges_count"]][traffic_summary_by_type.index != "external"].copy()

traffic_by_type = traffic_by_type.append(temp)
traffic_by_type.columns = ["traffic", "terminal_articles_count"]
traffic_by_type.index.name = "traffic_type"

traffic_by_type.loc["link", "terminal_articles_count"] = 2598302
traffic_by_type.loc["other", "terminal_articles_count"] = 414847

traffic_by_type

,traffic,terminal_articles_count
traffic_type,,
other-search,3034278822,3386425
other-empty,1779965188,5093433
other-internal,132349844,1348892
other-external,116732857,787311
other-other,36972380,374921
link,1718399222,2598302
other,60022438,414847


In [6]:
total_traffic = traffic_by_type.traffic.sum()
total_traffic

6878720751

In [7]:
traffic_by_type["traffic_pct"] = round(traffic_by_type.traffic / total_traffic * 100, 2)
traffic_by_type

,traffic,terminal_articles_count,traffic_pct
traffic_type,,,
other-search,3034278822,3386425,44.11
other-empty,1779965188,5093433,25.88
other-internal,132349844,1348892,1.92
other-external,116732857,787311,1.70
other-other,36972380,374921,0.54
link,1718399222,2598302,24.98
other,60022438,414847,0.87


From the [EDA notebook](English_Wikipedia_EDA.ipynb#How-many-articles-are-there-in-the-dataset?):  

total number of articles in the dataset = 5185699

In [8]:
total_articles_count = 5185699

In [9]:
traffic_by_type["terminal_articles_pct"] = round(traffic_by_type.terminal_articles_count 
                                                 / total_articles_count * 100, 2)

In [10]:
# add display labels for viz

traffic_by_type.loc["other-empty", "label"] = "empty referer"
traffic_by_type.loc["other-external", "label"] = "external websites"
traffic_by_type.loc["other-internal", "label"] = "other Wikimedia"
traffic_by_type.loc["other-other", "label"] = "unknown"
traffic_by_type.loc["other-search", "label"] = "search engines"
traffic_by_type.loc["link", "label"] = "links"
traffic_by_type.loc["other", "label"] = "internal search"

In [11]:
traffic_by_type

,traffic,terminal_articles_count,traffic_pct,terminal_articles_pct,label
traffic_type,,,,,
other-search,3034278822,3386425,44.11,65.30,search engines
other-empty,1779965188,5093433,25.88,98.22,empty referer
other-internal,132349844,1348892,1.92,26.01,other Wikimedia
other-external,116732857,787311,1.70,15.18,external websites
other-other,36972380,374921,0.54,7.23,unknown
link,1718399222,2598302,24.98,50.11,links
other,60022438,414847,0.87,8.00,internal search


In [12]:
len(traffic_by_type)

7

In [13]:
traffic_by_type.to_csv("traffic_by_type.csv")

In [14]:
!head traffic_by_type.csv

traffic_type,traffic,terminal_articles_count,traffic_pct,terminal_articles_pct,label
other-search,3034278822,3386425,44.11,65.3,search engines
other-empty,1779965188,5093433,25.88,98.22,empty referer
other-internal,132349844,1348892,1.92,26.01,other Wikimedia
other-external,116732857,787311,1.7,15.18,external websites
other-other,36972380,374921,0.54,7.23,unknown
link,1718399222,2598302,24.98,50.11,links
other,60022438,414847,0.87,8.0,internal search


Top 20 article listings by traffic volume

In [18]:
# Got an error when trying to unpickle the dict pickle below made on AWS.
# Solution source: https://github.com/pandas-dev/pandas/issues/16474#issuecomment-305566022 

import sys
# 1
import pandas.core.indexes 
sys.modules['pandas.indexes'] = pandas.core.indexes

In [19]:
# unpickle
with open(pickles_path + "en_1218_top20_by_traffic.pkl", 'rb') as picklefile: 
    top_traffic_nodes_dict = pickle.load(picklefile)

top_traffic_nodes_dict.keys()

dict_keys(['unknown_external_traffic', 'external_search_traffic', 'link_in_traffic', 'search_out_traffic', 'empty_referer_traffic', 'search_traffic_from_main_page', 'search_traffic_to_main_page', 'external_website_traffic', 'other_wikimedia_traffic', 'search_traffic_to_hyphen_minus', 'link_out_traffic', 'search_in_traffic'])

#### Get data for the external search top 20 articles list

In [20]:
top_traffic_nodes_dict["external_search_traffic"]

,title,link_in_degree,link_in_traffic,link_out_degree,link_out_traffic,search_in_degree,search_in_traffic,search_out_degree,search_out_traffic,external_search_traffic,external_website_traffic,empty_referer_traffic,other_wikimedia_traffic,unknown_external_traffic,search_traffic_from_main_page,search_traffic_to_hyphen_minus,search_traffic_to_main_page
0,George_H._W._Bush,956,1108189,811,4466936.0,227,5630.0,374,43016.0,4576854,38972,804462,109536,27129,220667.0,4171.0,23260.0
1,Jason_Momoa,91,639353,96,2226611.0,142,6451.0,180,45674.0,3538068,9764,990366,45797,16300,15162.0,2005.0,2346.0
2,2.0_(film),102,223635,206,1127380.0,267,23563.0,253,33927.0,3475113,19564,793408,44383,182358,13367.0,9132.0,3167.0
3,Bird_Box_(film),42,682992,67,2042949.0,328,10416.0,411,24767.0,3251996,7054,586351,127234,10447,33848.0,1956.0,2544.0
4,Main_Page,15,31170,0,NaN,0,NaN,0,NaN,3020671,677294,492341152,8826536,81647,NaN,NaN,NaN
5,Aquaman_(film),144,408421,250,2391941.0,710,34309.0,586,40158.0,2634665,9884,582462,131946,32533,52985.0,3903.0,2763.0
6,Bird_Box,8,200893,30,737163.0,7,192.0,45,30210.0,2328884,4305,105930,12491,4258,1732.0,825.0,909.0
7,Priyanka_Chopra,283,575481,423,1245686.0,81,3945.0,83,9872.0,2231176,5890,236831,23470,34612,6275.0,1802.0,1149.0
8,List_of_most-disliked_YouTube_videos,38,117115,105,741294.0,40,958.0,22,1433.0,2226602,30233,362723,26659,5876,5485.0,693.0,1602.0
9,Freddie_Mercury,366,336621,321,1155736.0,138,4161.0,133,18518.0,2050628,22290,440067,48257,8249,13117.0,1898.0,1829.0


In [33]:
top20_external_search_traffic = \
    top_traffic_nodes_dict["external_search_traffic"][["title", "external_search_traffic"]].copy()

top20_external_search_traffic.columns = ["title_raw", "traffic"]

# clean up the title
top20_external_search_traffic["title"] = \
    top20_external_search_traffic.title_raw.str.replace('_', ' ')

# add rank
top20_external_search_traffic["rank"] = top20_external_search_traffic.index + 1

#top20_external_search_traffic.drop("title_raw", axis=1, inplace=True)

top20_external_search_traffic

,title_raw,traffic,title,rank
0,George_H._W._Bush,4576854,George H. W. Bush,1
1,Jason_Momoa,3538068,Jason Momoa,2
2,2.0_(film),3475113,2.0 (film),3
3,Bird_Box_(film),3251996,Bird Box (film),4
4,Main_Page,3020671,Main Page,5
5,Aquaman_(film),2634665,Aquaman (film),6
6,Bird_Box,2328884,Bird Box,7
7,Priyanka_Chopra,2231176,Priyanka Chopra,8
8,List_of_most-disliked_YouTube_videos,2226602,List of most-disliked YouTube videos,9
9,Freddie_Mercury,2050628,Freddie Mercury,10


In [34]:
top20_external_search_traffic.to_csv("top20_external_search_traffic.tsv", sep="\t", index=False)

In [35]:
!head top20_external_search_traffic.tsv

title_raw	traffic	title	rank
George_H._W._Bush	4576854	George H. W. Bush	1
Jason_Momoa	3538068	Jason Momoa	2
2.0_(film)	3475113	2.0 (film)	3
Bird_Box_(film)	3251996	Bird Box (film)	4
Main_Page	3020671	Main Page	5
Aquaman_(film)	2634665	Aquaman (film)	6
Bird_Box	2328884	Bird Box	7
Priyanka_Chopra	2231176	Priyanka Chopra	8
List_of_most-disliked_YouTube_videos	2226602	List of most-disliked YouTube videos	9


In [36]:

top_traffic_nodes_dict.keys()

dict_keys(['unknown_external_traffic', 'external_search_traffic', 'link_in_traffic', 'search_out_traffic', 'empty_referer_traffic', 'search_traffic_from_main_page', 'search_traffic_to_main_page', 'external_website_traffic', 'other_wikimedia_traffic', 'search_traffic_to_hyphen_minus', 'link_out_traffic', 'search_in_traffic'])

#### Get data for the empty referer top 20 articles list

In [38]:
# external_search_traffic is already done
traffic_types_list = ['empty_referer_traffic', 'external_website_traffic', 
                      'other_wikimedia_traffic', 'unknown_external_traffic',
                      'link_in_traffic', 'search_in_traffic']

for tt in traffic_types_list:
    top20_traffic = top_traffic_nodes_dict[tt][["title", tt]].copy()
    top20_traffic.columns = ["title_raw", "traffic"]

    # clean up the title
    top20_traffic["title"] = top20_traffic.title_raw.str.replace('_', ' ')

    # add rank
    top20_traffic["rank"] = top20_traffic.index + 1

    # output to tsv (not csv since article titles can have commas in them)
    top20_traffic.to_csv("top20_" + tt + ".tsv", sep="\t", index=False)

In [39]:
!head top20_empty_referer_traffic.tsv

title_raw	traffic	title	rank
Main_Page	492341152	Main Page	1
Hyphen-minus	15498618	Hyphen-minus	2
XHamster	4281194	XHamster	3
Darth_Vader	1801074	Darth Vader	4
Segersjön	1724261	Segersjön	5
Lists_of_deaths_by_year	1457060	Lists of deaths by year	6
Exo_(band)	1223507	Exo (band)	7
2018_FIFA_World_Cup	1077101	2018 FIFA World Cup	8
Jason_Momoa	990366	Jason Momoa	9
